In [ ]:
# To confirm you have installed the latest version of PixieDust on your system, run this cell
!pip install --ignore-installed --upgrade pixiedust

In [ ]:
import pixiedust

In [ ]:
!pip list --format=columns

In [ ]:
def readDataFrameFromCloudant(database):
    
    cloudantdata=spark.read.load(database, "org.apache.bahir.cloudant")
    
    cloudantdata.createOrReplaceTempView("historicaldata1")
    spark.sql("SELECT * from historicaldata1").show()
    return cloudantdata

In [ ]:
# Credentials from your cloudantNoSQLDB service
hostname = "<your cloudant hostname credential here -bluemix.cloudant.com>"
user = "<your cloudant user credential here -bluemix>"
pw = "<Your cloudant password credential here>"

database = "historicaldata1" #as long as you didn't change this in the NodeRED flow the database name stays the same

In [ ]:
spark = SparkSession \
    .builder\
    .appName("Cloudant Spark SQL Example in Python using temp tables")\
    .config("cloudant.host",hostname)\
    .config("cloudant.username", user) \
    .config("cloudant.password", pw) \
    .getOrCreate()
    
cloudantdata=readDataFrameFromCloudant(database)

In [ ]:
display(cloudantdata)

In [ ]:
df = spark.sql("select avg(temp), std(temp), max(temp), min(temp) from historicaldata1")
df.show()
display(df)

In [ ]:
# Calculate and show the mean and standard deviation
df = spark.sql("select mean(temp) as mean, std(temp) as std from (SELECT ROW_NUMBER() OVER (ORDER BY time desc) AS row, * FROM historicaldata1 where temp is not null) where row < 30 ")
df.show()

# Calculate the zscores
# A z-score is the number of standard deviations from the mean a data point is. 
# But more technically it's a measure of how many standard deviations below or above the population mean a raw score is.
# A z-score is also known as a standard score and it can be placed on a normal distribution curve.
mean = df.first().mean
std = df.first().std
spark.sql("SELECT * from (select abs( ({0}-temp)/({1}+0.0001) ) as zscore, temp,ROW_NUMBER() OVER (ORDER BY time desc) AS row FROM historicaldata1 where temp is not null order by time desc) where row < 20".format(mean,std)).show()

# Calculate the Average z-scores
df = spark.sql("SELECT mean(zscore) as avgzscore from (select abs( ({0}-temp)/({1}+0.0001) ) as zscore, temp,ROW_NUMBER() OVER (ORDER BY time desc) AS row FROM historicaldata1 where temp is not null order by time desc) where row < 20".format(mean,std))
df.show()

In [ ]:
zscore = df.first().avgzscore